In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import sys
import os

# Añadir el directorio raíz del proyecto al path
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

from src.db_manager import SQLiteWrapper
from src.transfer_annotations import transfer_all_annotations

pd.options.display.max_columns = None

## 1. Configuración de Bases de Datos

Especifica las rutas a tus bases de datos origen (antigua) y destino (nueva).

In [2]:
# Base de datos ANTIGUA (origen de las anotaciones)
db_path_old = os.path.join("..", "data", "KoboReader.sqlite")

# Base de datos NUEVA (destino de las anotaciones)
db_path_new = os.path.join("..", "data", "KoboReader_5ene.sqlite")

# Conectar a ambas bases de datos
db_old = SQLiteWrapper(db_path_old)
db_old.connect()

db_new = SQLiteWrapper(db_path_new)
db_new.connect()

print(f"✅ Conectado a BD antigua: {db_path_old}")
print(f"✅ Conectado a BD nueva: {db_path_new}")

✅ Conectado a BD antigua: ..\data\KoboReader.sqlite
✅ Conectado a BD nueva: ..\data\KoboReader_5ene.sqlite


## 2. Opción A: Transferir un libro específico

Útil cuando has redescargado un solo libro o unos pocos libros.

In [ ]:
# Transferir un libro específico usando transfer_all_annotations con filtro
# Cambiar el título exacto del libro que necesites

book_title = "Una historia de España"  # Título EXACTO del libro

# Primero simular (dry_run=True)
stats = transfer_all_annotations(
    source_db_path=db_path_old,
    target_db_path=db_path_new,
    dry_run=True,  # Cambiar a False para ejecutar realmente
    book_filter=[book_title]  # Lista con el/los libro(s) específico(s)
)

print("\n" + "="*80)
print("💡 Si todo se ve correcto, cambia dry_run=False y ejecuta de nuevo")
print("="*80)

## 3. Opción B: Transferir todos los libros automáticamente

Transfiere todos los libros que tienen anotaciones en la BD antigua pero no en la nueva.

**⚠️ IMPORTANTE**: Siempre ejecuta primero con `dry_run=True` para ver qué se va a transferir.

In [5]:
# Transferencia masiva de todos los libros

stats = transfer_all_annotations(
    source_db_path=db_path_old,
    target_db_path=db_path_new,
    dry_run=False,  # ⚠️ Cambiar a False para ejecutar realmente
    # dry_run=True,  # ⚠️ Cambiar a False para ejecutar realmente
    book_filter=None  # O especificar: ["Libro 1", "Libro 2"]
    # book_filter= [
    #     "The Slight Edge: Turning Simple Disciplines into Massive Success and Happiness",
    #     "Una historia de España"
    #     ]
  # O especificar: ["Libro 1", "Libro 2"]
)

print("\n" + "="*80)
print("💡 Si todo se ve correcto, cambia dry_run=False y ejecuta de nuevo")
print("="*80)

TRANSFERENCIA MASIVA DE ANOTACIONES
Modo: ⚠️ EJECUCIÓN REAL

Origen: ..\data\KoboReader.sqlite
Destino: ..\data\KoboReader_5ene.sqlite

📚 Libros con anotaciones en BD origen: 52
📗 Libros con anotaciones en BD destino: 29

PROCESANDO LIBROS

[1/52] 12 reglas para vivir
   Autor: Jordan Peterson
   Anotaciones: 12
   ⏭️  OMITIDO: Ya existen anotaciones en BD destino

[2/52] 1984
   Autor: George Orwell
   Anotaciones: 2
   ⏭️  OMITIDO: Ya existen anotaciones en BD destino

[3/52] A puerta cerrada
   Autor: J. D. Barker
   Anotaciones: 1

📚 Libros con anotaciones en BD origen: 52
📗 Libros con anotaciones en BD destino: 29

PROCESANDO LIBROS

[1/52] 12 reglas para vivir
   Autor: Jordan Peterson
   Anotaciones: 12
   ⏭️  OMITIDO: Ya existen anotaciones en BD destino

[2/52] 1984
   Autor: George Orwell
   Anotaciones: 2
   ⏭️  OMITIDO: Ya existen anotaciones en BD destino

[3/52] A puerta cerrada
   Autor: J. D. Barker
   Anotaciones: 1
   ✅ TRANSFERIDO: 1 anotaciones

[4/52] Acontece que 

DatabaseError: Execution failed on sql '
        SELECT COUNT(*) as total
        FROM Bookmark
        WHERE VolumeID = 'file:///mnt/onboard/.kobo/dropbox/Surrounded by idiots_ the four types of human behavior and -- Bradbury, Rod;Erikson, Thomas;Pender, Martin -- St_ Martin's Essentials, First Edition, -- 9781250179944 -- d5276a97a2db0fa3010cac81bdeb6ac3 -- Anna’s Archive.epub'
        AND Type IN ('highlight', 'note')
    ': near "s": syntax error

### ⚠️ Importante sobre el guardado

El script **SÍ guarda en la base de datos** cuando `dry_run=False`, PERO:

1. **Omite libros que ya tienen anotaciones** en la BD destino (previene duplicados)
2. Si el output dice "OMITIDO: Ya existen anotaciones", entonces NO se transfirió nada
3. El commit se hace automáticamente después de transferir cada libro exitosamente

**Para forzar la transferencia:**
- Debes **borrar primero** las anotaciones existentes del libro en la BD destino
- O usar la celda de abajo para verificar si realmente se guardaron los cambios

In [11]:
# Verificar si realmente se guardaron las anotaciones en la BD nueva
# Esta celda verifica el número REAL de anotaciones en el archivo físico

# Reconectar para asegurar que leemos el estado actual del archivo
db_new.close()
db_new = SQLiteWrapper(db_path_new)
db_new.connect()

# Contar anotaciones actuales
check_query = """
    SELECT 
        c.Title as Título,
        COUNT(*) as Anotaciones_Actuales
    FROM Bookmark b
    INNER JOIN content c ON b.VolumeID = c.ContentID
    WHERE b.Type IN ('highlight', 'note')
    AND c.Title IN ('Una historia de España', 'The Slight Edge: Turning Simple Disciplines into Massive Success and Happiness')
    GROUP BY c.Title
"""

current_annotations = db_new.get_query_df(check_query)
print("="*80)
print("📊 VERIFICACIÓN: Anotaciones en BD nueva (estado actual)")
print("="*80)
print(current_annotations.to_string(index=False))

print("\n💡 Si NO aparecen anotaciones:")
print("   1. Los libros fueron OMITIDOS porque ya tenían anotaciones")
print("   2. O hubo un error durante la transferencia")
print("\n💡 Si aparecen anotaciones:")
print("   ✅ La transferencia fue exitosa y se guardó en el archivo")

📊 VERIFICACIÓN: Anotaciones en BD nueva (estado actual)
                                                                        Título  Anotaciones_Actuales
The Slight Edge: Turning Simple Disciplines into Massive Success and Happiness                    14
                                                        Una historia de España                     3

💡 Si NO aparecen anotaciones:
   1. Los libros fueron OMITIDOS porque ya tenían anotaciones
   2. O hubo un error durante la transferencia

💡 Si aparecen anotaciones:
   ✅ La transferencia fue exitosa y se guardó en el archivo


## 4. Verificación: Ver libros con anotaciones

Útil para identificar qué libros necesitan transferencia.

In [7]:
# Libros con anotaciones en BD ANTIGUA
query_old = """
    SELECT 
        c.Title as Título,
        c.Attribution as Autor,
        COUNT(*) as Anotaciones
    FROM Bookmark b
    INNER JOIN content c ON b.VolumeID = c.ContentID
    WHERE b.Type IN ('highlight', 'note')
    GROUP BY c.Title, c.Attribution
    ORDER BY Anotaciones DESC
"""

books_old = db_old.get_query_df(query_old)
print("="*80)
print(f"📚 LIBROS CON ANOTACIONES EN BD ANTIGUA: {len(books_old)}")
print("="*80)
print(books_old.head(20))

📚 LIBROS CON ANOTACIONES EN BD ANTIGUA: 52
                                               Título  \
0   ES MANIPULACIÓN Y NO LO SABES: DESACTIVA LAS T...   
1                            The Happiness Hypothesis   
2   Charisma on Command: Inspire, Impress, and Ene...   
3                                        Fluir (Flow)   
4                                Surrounded by Idiots   
5   Cómo analizar a las personas y el lenguaje cor...   
6                                              Models   
7   Attached: Are you Anxious, Avoidant or Secure?...   
8                                     El arte de amar   
9   Autopsicología. Ejercicios y claves para una b...   
10                             Las 5 trampas del amor   
11  El sutil arte de que (casi todo) te importe un...   
12                                            Mindset   
13                       Los dones de la imperfección   
14                               No More Mr. Nice Guy   
15                                           

In [8]:
# Libros con anotaciones en BD NUEVA
query_new = """
    SELECT 
        c.Title as Título,
        c.Attribution as Autor,
        COUNT(*) as Anotaciones
    FROM Bookmark b
    INNER JOIN content c ON b.VolumeID = c.ContentID
    WHERE b.Type IN ('highlight', 'note')
    GROUP BY c.Title, c.Attribution
    ORDER BY Anotaciones DESC
"""

books_new = db_new.get_query_df(query_new)
print("="*80)
print(f"📗 LIBROS CON ANOTACIONES EN BD NUEVA: {len(books_new)}")
print("="*80)
print(books_new.head(20))

📗 LIBROS CON ANOTACIONES EN BD NUEVA: 46
                                               Título  \
0   ES MANIPULACIÓN Y NO LO SABES: DESACTIVA LAS T...   
1                            The Happiness Hypothesis   
2   Charisma on Command: Inspire, Impress, and Ene...   
3                                        Fluir (Flow)   
4                                Surrounded by Idiots   
5   Cómo analizar a las personas y el lenguaje cor...   
6                                              Models   
7   Attached: Are you Anxious, Avoidant or Secure?...   
8                                     El arte de amar   
9   Autopsicología. Ejercicios y claves para una b...   
10                             Las 5 trampas del amor   
11  El sutil arte de que (casi todo) te importe un...   
12                                            Mindset   
13                       Los dones de la imperfección   
14                               No More Mr. Nice Guy   
15                                             

In [ ]:
# Identificar libros que NECESITAN transferencia:
# Son aquellos que:
# 1. Tienen anotaciones en BD antigua
# 2. Existen en BD nueva (fueron redescargados)
# 3. Las anotaciones en BD antigua NO tienen capítulos (ContentID no mapeado)

print("="*80)
print("📋 ANALIZANDO LIBROS QUE NECESITAN TRANSFERENCIA")
print("="*80)

# Obtener libros que existen en ambas BDs
titles_old = set(books_old['Título'].tolist())
titles_new = set(books_new['Título'].tolist())

# Libros que están en ambas (redescargados)
books_in_both = titles_old & titles_new

print(f"\n📚 Libros en AMBAS bases de datos: {len(books_in_both)}")

if books_in_both:
    # Para cada libro en ambas BDs, verificar si tiene problemas de capítulos en BD antigua
    books_needing_transfer = []
    
    for book_title in books_in_both:
        # Obtener VolumeID del libro en BD antigua
        volume_query = """
            SELECT DISTINCT VolumeID
            FROM Bookmark b
            INNER JOIN content c ON b.VolumeID = c.ContentID
            WHERE c.Title = ?
            LIMIT 1
        """
        volume_result = db_old.get_query_df(volume_query, params=(book_title,))
        
        if len(volume_result) > 0:
            volume_id = volume_result['VolumeID'].iloc[0]
            
            # Verificar cuántas anotaciones tienen ContentID sin mapear a capítulo
            # (anotaciones donde ContentID = VolumeID o no existe en content)
            problem_query = """
                SELECT COUNT(*) as sin_capitulo
                FROM Bookmark b
                LEFT JOIN content c ON b.ContentID = c.ContentID
                WHERE b.VolumeID = ?
                AND b.Type IN ('highlight', 'note')
                AND (c.ContentID IS NULL OR c.ContentType != 9)
            """
            problem_result = db_old.get_query_df(problem_query, params=(volume_id,))
            
            total_query = """
                SELECT COUNT(*) as total
                FROM Bookmark b
                WHERE b.VolumeID = ?
                AND b.Type IN ('highlight', 'note')
            """
            total_result = db_old.get_query_df(total_query, params=(volume_id,))
            
            sin_capitulo = problem_result['sin_capitulo'].iloc[0]
            total = total_result['total'].iloc[0]
            
            # Si todas o la mayoría de anotaciones no tienen capítulo, necesita transferencia
            if sin_capitulo > 0 and sin_capitulo == total:
                books_needing_transfer.append({
                    'Título': book_title,
                    'Anotaciones': total,
                    'Sin_Capítulos': sin_capitulo
                })
    
    if books_needing_transfer:
        transfer_df = pd.DataFrame(books_needing_transfer).sort_values('Anotaciones', ascending=False)
        print(f"\n⚠️ LIBROS QUE NECESITAN TRANSFERENCIA: {len(books_needing_transfer)}")
        print("   (Libros redescargados donde las anotaciones antiguas NO tienen capítulos)")
        print("\n" + "="*80)
        print(transfer_df.to_string(index=False))
    else:
        print("\n✅ No hay libros que necesiten transferencia")
        print("   Todos los libros redescargados ya tienen anotaciones con capítulos correctos")
else:
    print("\n✅ No hay libros en común entre ambas bases de datos")

📋 ANALIZANDO LIBROS QUE NECESITAN TRANSFERENCIA

📚 Libros en AMBAS bases de datos: 46

⚠️ LIBROS QUE NECESITAN TRANSFERENCIA: 17
   (Libros redescargados donde las anotaciones antiguas NO tienen capítulos)

                                                                        Título  Anotaciones  Sin_Capítulos
                                                      The Happiness Hypothesis          260            260
                                                          Surrounded by Idiots          158            158
                                                                        Models          154            154
                                                        Las 5 trampas del amor           92             92
                                                                       Mindset           79             79
                                                  Los dones de la imperfección           76             76
                                            

### 💡 Explicación

La celda anterior identifica libros que **necesitan transferencia de anotaciones** porque:

1. ✅ El libro existe en **ambas** bases de datos (fue redescargado)
2. ⚠️ Las anotaciones en la BD **antigua** NO tienen capítulos correctamente mapeados
3. ✅ En la BD **nueva** el libro tiene la estructura de capítulos correcta (96 capítulos, por ejemplo)

**Por qué transferir:**
- Las anotaciones antiguas están vinculadas al libro completo (sin capítulos específicos)
- Al redescargarlo, el Kobo indexó correctamente los capítulos
- Transferir las anotaciones permite mapearlas a los capítulos correctos usando la nueva estructura

## 5. Verificación Post-Transferencia

Después de ejecutar la transferencia, verifica que las anotaciones se copiaron correctamente.

In [8]:
# Verificar anotaciones de un libro específico en la BD nueva
# Cambia el patrón según el libro que transferiste

book_to_verify = '%historia de España%'

anotaciones_verificadas = db_new.get_query_df(f"""
    SELECT 
        l.Author as Autor,
        l.Title as Título, 
        COALESCE(c.Title, l.Title) as Capítulo,
        b.Text as Texto, 
        b.DateCreated as 'Fecha de creación'
    FROM Bookmark b 
    LEFT JOIN content c ON b.ContentID = c.ContentID
    INNER JOIN (
        SELECT DISTINCT b.VolumeID, c.Title, c.Attribution as Author
        FROM Bookmark b 
        INNER JOIN content c ON b.VolumeID = c.ContentID
    ) l ON b.VolumeID = l.VolumeID
    WHERE b.Type IN ("highlight", "note")
    AND l.Title LIKE '{book_to_verify}'
""")

print("="*80)
print(f"✅ ANOTACIONES TRANSFERIDAS: {len(anotaciones_verificadas)}")
print("="*80)
print(anotaciones_verificadas[['Capítulo', 'Texto', 'Fecha de creación']].to_string(index=False))

✅ ANOTACIONES TRANSFERIDAS: 0
Empty DataFrame
Columns: [Capítulo, Texto, Fecha de creación]
Index: []


## 📝 Notas Importantes

### ✅ Qué hace el script:
- Copia anotaciones (highlights y notas) de una BD a otra
- Intenta mapear capítulos correctamente
- Genera nuevos IDs únicos para evitar conflictos
- Preserva fechas, textos y posiciones originales
- NO duplica anotaciones que ya existen

### ⚠️ Limitaciones:
- Busca libros por título exacto (debe coincidir)
- Si un libro cambió de nombre al redescargarse, no lo encontrará
- Anotaciones de formatos .kepub antiguos pueden no mapear capítulos correctamente

### 🔧 Alternativa: Script por línea de comandos

También puedes usar el script desde terminal:

```bash
# Simular
python src/transfer_annotations.py --source data/KoboReader.sqlite --target data/KoboReader_5ene.sqlite --dry-run

# Ejecutar
python src/transfer_annotations.py --source data/KoboReader.sqlite --target data/KoboReader_5ene.sqlite
```

Ver `README_TRANSFER.md` para más detalles.